In [180]:
import os 
os.getcwd()

# assign the path with location of data to this variable before running the following functions
data_path = "Replace with path to csv files"

import numpy as np

from os import listdir
from os.path import isfile, join
csv_files = [f for f in listdir(data_path) if isfile(join(data_path, f)) and f.endswith(".csv")]

# print list of .csv files found in the path given
csv_files

['37 Field Extract_11 Sept CSV.csv',
 '37 Field Extract_30 July 2020 CSV.csv',
 '37 Field Extract_30 April 2020.csv',
 '37 Field Extract_27 August 2020 CSV.csv',
 '37 Field Extract_14 May 2020.csv',
 '37 Field Extract_20 Feb 2020 CSV.csv',
 '37 Field Extract_27 Jan 2020 CSV.csv',
 '37 Field Extract_25 June 2020.csv',
 '37 Field Extract_11 March 2020.csv']

In [182]:
from datetime import date
import dateutil.parser

# start and end dates for data extraction
start = date(2020, 1, 1)
end = date(2020, 12, 31)

# data cleanup func to remove any braces and convert data to float
def consumption_format_helper(x):
    return x if type(x).__name__ == 'float' else float(x.strip("()"))

# parse date into required format
def date_helper(current_date):
    return dateutil.parser.parse(str(current_date)).date()


def transform_data(df):
    
    # sort values based on READ_DATE in descending order. This gives the latest reading. 
    # groupby PROPERTY_KEY and POD_NO to get value for each meter. 
    # extract ['PROPERTY_KEY', 'POD_NO', 'CONSUMPTION_TYPE', 'READ_DATE', 'DAILY_AVG_CONSUMPTION'] as resultant DataFrame
    temp = df.sort_values(['READ_DATE'],ascending=False).groupby(['PROPERTY_KEY', 'POD_NO']).head(1)[['PROPERTY_KEY', 'POD_NO', 'CONSUMPTION_TYPE', 'READ_DATE', 'DAILY_AVG_CONSUMPTION']]
    
    # convert READ_DATE values to String if not null (represented by nan in df)
    temp = temp[temp['READ_DATE'].map(lambda x: str(x) != 'nan')]
    
    # filter out readings for meters of CONSUMPTION_TYPE=Zone Meter
    temp = temp[temp['CONSUMPTION_TYPE'].map(lambda x: str(x) != 'Zone Meter')]
    
    # convert READ_DATE to compatible format
    temp['READ_DATE'] = temp.apply(lambda row : date_helper(row['READ_DATE']), axis = 1)
    
    # filter out meter readings outside the time period specified by start and end dates
    temp = temp[(temp['READ_DATE'] >= start) & (temp['READ_DATE'] <= end )]
    
    # create new field MONTLY_CONSUMPTION as 30 x DAILY_AVG_CONSUMPTION for a given meter
    temp['MONTHLY_CONSUMPTION'] = temp.apply(lambda row : 30 * consumption_format_helper(row['DAILY_AVG_CONSUMPTION']), axis = 1)
    
    return temp

In [185]:
import pandas as pd
from collections import defaultdict

months_dict = defaultdict(int)

print("Started Processing files in folder: " + data_path + "\n")

# go through each csv file in data folder and call transform_data func
for file in csv_files:
    print("Processing file: {}\n".format(file))
    df = pd.read_csv(join(data_path, file))
    df.dropna()
    df = transform_data(df)
    for index, row in df.iterrows():
        current_date = dateutil.parser.parse(str(row['READ_DATE']))
        current_date = current_date.strftime("%b %Y")
        temp = str(current_date) + "[SEP]" + str(row['CONSUMPTION_TYPE'])
        months_dict[temp] += row['MONTHLY_CONSUMPTION']

data = []
            
for key in months_dict.keys():
    temp = key.split('[SEP]')
    data.append((temp[0], temp[1], months_dict[key]))

# write output dict into .csv called consumption.csv
print("Writing data into csv file")
result_df = pd.DataFrame(data=data, columns=['Month', 'Type', 'Consumption'])
result_df.to_csv('consumption.csv',index=False)
        

Started Processing files in folder: /Users/sujaygd/Desktop/ASU CIC Part time/Durban/Data/RMS Data/2020

Processing file: 37 Field Extract_11 Sept CSV.csv

Processing file: 37 Field Extract_30 July 2020 CSV.csv

Processing file: 37 Field Extract_30 April 2020.csv

Processing file: 37 Field Extract_27 August 2020 CSV.csv

Processing file: 37 Field Extract_14 May 2020.csv



/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,10,20,21,25,26,37,47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processing file: 37 Field Extract_20 Feb 2020 CSV.csv

Processing file: 37 Field Extract_27 Jan 2020 CSV.csv

Processing file: 37 Field Extract_25 June 2020.csv

Processing file: 37 Field Extract_11 March 2020.csv



/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,26,47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Writing data into csv file
